<a href="https://colab.research.google.com/github/mattiadido95/Haze-Fog-suppression/blob/main/pix2pixHD_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Libraries

In [1]:
!pip install dominate gdown

In [2]:
from google.colab import drive
import os
import gdown

In [3]:
# mount drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [4]:
folder_path = "drive/MyDrive/Haze-Fog-suppression"
os.chdir(folder_path)

# Download library for no reference metrics

In [ ]:
!git clone https://github.com/buyizhiyou/NRVQA.git

Cloning into 'NRVQA'...
remote: Enumerating objects: 166, done.
remote: Counting objects: 100% (166/166), done.
remote: Compressing objects: 100% (137/137), done.
remote: Total 166 (delta 43), reused 131 (delta 22), pack-reused 0
Receiving objects: 100% (166/166), 9.97 MiB | 11.47 MiB/s, done.
Resolving deltas: 100% (43/43), done.


# Install Apex for Automatic Mixed Precision to speed up training (NOT WORKING)

In [ ]:
#!git clone https://github.com/NVIDIA/apex
%cd apex
# if pip >= 23.1 (ref: https://pip.pypa.io/en/stable/news/#v23-1) which supports multiple `--config-settings` with the same key...
!pip install -v --disable-pip-version-check --no-cache-dir --no-build-isolation --config-settings "--build-option=--cpp_ext" --config-settings "--build-option=--cuda_ext" ./
%cd ..

/content/drive/MyDrive/Haze-Fog-suppression/pix2pixHD/apex
Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
Processing /content/drive/MyDrive/Haze-Fog-suppression/pix2pixHD/apex
  Running command Preparing metadata (pyproject.toml)


  torch.__version__  = 2.0.1+cu118


  running dist_info
  creating /tmp/pip-modern-metadata-k24uqvr5/apex.egg-info
  writing /tmp/pip-modern-metadata-k24uqvr5/apex.egg-info/PKG-INFO
  writing dependency_links to /tmp/pip-modern-metadata-k24uqvr5/apex.egg-info/dependency_links.txt
  writing requirements to /tmp/pip-modern-metadata-k24uqvr5/apex.egg-info/requires.txt
  writing top-level names to /tmp/pip-modern-metadata-k24uqvr5/apex.egg-info/top_level.txt
  writing manifest file '/tmp/pip-modern-metadata-k24uqvr5/apex.egg-info/SOURCES.txt'
  reading manifest file '/tmp/pip-modern-metadata-k24uqvr5/apex.egg-info/SOURCES.txt'
  adding license file 'LICENSE'
  writing manifest file '/tmp/pip-modern-metadata-k24uqvr5/apex.egg-inf

# Utility

In [28]:
import cv2
import numpy as np
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr
import pandas as pd
import subprocess

# salva in pix2pixHD/results le metriche
def calculate_metrics(synth_folder, gt_folder,type):
    # Ottieni la lista dei nomi di file dalle cartelle

    image_files = os.listdir(synth_folder)

    # Filtra i nomi dei file per rimuovere quelli che contengono "input_label"
    filtered_image_files = [file for file in image_files if "input_label" not in file]

    # Inizializza liste per memorizzare i risultati
    mse_scores = []
    ssim_scores = []
    psnr_scores = []

    piqe_scores = []
    niqe_scores = []

    # Itera attraverso le immagini e calcola le metriche
    for image_file in filtered_image_files:
        # Carica le immagini
        synth_image = cv2.imread(os.path.join(synth_folder, image_file))
        gt_image = cv2.imread(os.path.join(gt_folder, image_file[:4]+".png"))

        # Ridimensiona l'immagine sintetizzata alla stessa dimensione dell'immagine originale
        synth_image = cv2.resize(synth_image, (gt_image.shape[1], gt_image.shape[0]))

        # Calcola le metriche con skimage
        mse = np.mean((gt_image - synth_image) ** 2)
        ssim_score = ssim(gt_image, synth_image, channel_axis=2)
        psnr_score = psnr(gt_image, synth_image)

        # Calcola le metriche con piqe e niqe (high score means low quality)
        synth_array = synth_image / 255.0

        # Esegui il comando e cattura l'output
        completed_process = subprocess.run(['python', '/content/drive/MyDrive/Haze-Fog-suppression/NRVQA/test.py', '--mode', 'piqe', f'--path={os.path.join(synth_folder, image_file)}'], stdout=subprocess.PIPE, text=True)

        # Ottieni l'output dall'oggetto CompletedProcess
        output = completed_process.stdout

        # Estrai lo score dalla stringa di output
        score_line = [line for line in output.splitlines() if 'score:' in line][0]
        piqe_score = float(score_line.split(':')[-1])

        # Esegui il comando e cattura l'output
        completed_process = subprocess.run(['python', '/content/drive/MyDrive/Haze-Fog-suppression/NRVQA/test.py', '--mode', 'niqe', f'--path={os.path.join(synth_folder, image_file)}'], stdout=subprocess.PIPE, text=True)

        # Ottieni l'output dall'oggetto CompletedProcess
        output = completed_process.stdout

        # Estrai lo score dalla stringa di output
        score_line = [line for line in output.splitlines() if 'score:' in line][0]
        niqe_score = float(score_line.split(':')[-1])

        # Aggiungi i risultati alle liste
        mse_scores.append(mse)
        ssim_scores.append(ssim_score)
        psnr_scores.append(psnr_score)
        piqe_scores.append(piqe_score)
        niqe_scores.append(niqe_score)


    # Creazione del dataframe
    data = {
        'Image': filtered_image_files,  # Nomi dei file delle immagini generate
        'MSE': mse_scores,
        'SSIM': ssim_scores,
        'PSNR': psnr_scores,
        'PIQE': piqe_scores,
        'NIQE': niqe_scores
    }

    df = pd.DataFrame(data)

    # Calcolo dei valori medi
    average_scores = df[['MSE', 'SSIM', 'PSNR', 'PIQE', 'NIQE']].mean()

    # Aggiunta della riga con i valori medi al dataframe
    average_scores_df = pd.DataFrame([average_scores], columns=['MSE', 'SSIM', 'PSNR', 'PIQE', 'NIQE'])
    average_scores_df['Image'] = 'AVG'
    # Aggiungi una riga vuota tra i risultati delle immagini e i valori medi
    df = pd.concat([df, pd.DataFrame(columns=df.columns)], ignore_index=True)
    df = pd.concat([df, average_scores_df], ignore_index=True)

    # Salva il dataframe completo in un file CSV
    df.to_csv(type + '_result.csv', index=False)

def test_indoor_outdoor(indoor_synth_folder, indoor_original_folder, outdoor_synth_folder, outdoor_original_folder, type):
  calculate_metrics(indoor_synth_folder, indoor_original_folder,type + "_indoor")
  calculate_metrics(outdoor_synth_folder, outdoor_original_folder,type + "_outdoor")

# Pix2PixHD download


In [ ]:
!git clone https://github.com/NVIDIA/pix2pixHD.git
os.chdir("pix2pixHD")

Cloning into 'pix2pixHD'...
remote: Enumerating objects: 340, done.
remote: Total 340 (delta 0), reused 0 (delta 0), pack-reused 340
Receiving objects: 100% (340/340), 55.68 MiB | 20.18 MiB/s, done.
Resolving deltas: 100% (156/156), done.
Updating files: 100% (115/115), done.
/content/drive/MyDrive/Haze-suppression/pix2pixHD


In [ ]:
os.makedirs("./checkpoints/label2city_1024p/")
os.chdir("./checkpoints/label2city_1024p/")

In [ ]:
url = 'https://drive.google.com/u/0/uc?id=1h9SykUnuZul7J3Nbms2QGH1wa85nbN2-&export=download'
output = 'latest_net_G.pth'
gdown.download(url, output, quiet=False)

Downloading...
From: https://drive.google.com/u/0/uc?id=1h9SykUnuZul7J3Nbms2QGH1wa85nbN2-&export=download
To: /content/drive/MyDrive/Haze-Fog-suppression/pix2pixHD/checkpoints/label2city_1024p/latest_net_G.pth
100%|██████████| 732M/732M [00:11<00:00, 62.1MB/s]


'latest_net_G.pth'

In [ ]:
os.chdir("../..")

# Pix2PixHD Train


In [ ]:
%cd pix2pixHD/

/content/drive/.shortcut-targets-by-id/1MeMU7hWD5WVMk25vU3ch5Ii3csnSydwf/Haze-Fog-suppression/pix2pixHD


## Unique model for different level of fog

In [ ]:
#training on low resolution(512) only G1 (global net)
!python train.py --continue_train --label_nc 0 --no_instance --name nebbia --dataroot ./datasets/nebbia --resize_or_crop crop --fineSize 512 --batchSize 4

------------ Options -------------
batchSize: 4
beta1: 0.5
checkpoints_dir: ./checkpoints
continue_train: True
data_type: 32
dataroot: ./datasets/nebbia
debug: False
display_freq: 100
display_winsize: 512
feat_num: 3
fineSize: 512
fp16: False
gpu_ids: [0]
input_nc: 3
instance_feat: False
isTrain: True
label_feat: False
label_nc: 0
lambda_feat: 10.0
loadSize: 1024
load_features: False
load_pretrain: 
local_rank: 0
lr: 0.0002
max_dataset_size: inf
model: pix2pixHD
nThreads: 2
n_blocks_global: 9
n_blocks_local: 3
n_clusters: 10
n_downsample_E: 4
n_downsample_global: 4
n_layers_D: 3
n_local_enhancers: 1
name: nebbia
ndf: 64
nef: 16
netG: global
ngf: 64
niter: 100
niter_decay: 100
niter_fix_global: 0
no_flip: False
no_ganFeat_loss: False
no_html: False
no_instance: True
no_lsgan: False
no_vgg_loss: False
norm: instance
num_D: 2
output_nc: 3
phase: train
pool_size: 0
print_freq: 100
resize_or_crop: crop
save_epoch_freq: 10
save_latest_freq: 1000
serial_batches: False
tf_log: False
use_dropou

In [ ]:
#training on high resolution (1024) G1 and G2 (local+global) starting from epoch 100 of G1

#import torch
#os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:512'
#torch.cuda.empty_cache()

!python train.py --netG local --continue_train --load_pretrain checkpoints/nebbia/ --label_nc 0 --no_instance --name nebbia --dataroot ./datasets/nebbia --resize_or_crop crop --fineSize 1024 --batchSize 2

------------ Options -------------
batchSize: 2
beta1: 0.5
checkpoints_dir: ./checkpoints
continue_train: True
data_type: 32
dataroot: ./datasets/nebbia
debug: False
display_freq: 100
display_winsize: 512
feat_num: 3
fineSize: 1024
fp16: False
gpu_ids: [0]
input_nc: 3
instance_feat: False
isTrain: True
label_feat: False
label_nc: 0
lambda_feat: 10.0
loadSize: 1024
load_features: False
load_pretrain: checkpoints/nebbia/
local_rank: 0
lr: 0.0002
max_dataset_size: inf
model: pix2pixHD
nThreads: 2
n_blocks_global: 9
n_blocks_local: 3
n_clusters: 10
n_downsample_E: 4
n_downsample_global: 4
n_layers_D: 3
n_local_enhancers: 1
name: nebbia
ndf: 64
nef: 16
netG: local
ngf: 64
niter: 100
niter_decay: 100
niter_fix_global: 0
no_flip: False
no_ganFeat_loss: False
no_html: False
no_instance: True
no_lsgan: False
no_vgg_loss: False
norm: instance
num_D: 2
output_nc: 3
phase: train
pool_size: 0
print_freq: 100
resize_or_crop: crop
save_epoch_freq: 10
save_latest_freq: 1000
serial_batches: False
tf_lo

## 3 GANS for each fog level

In [ ]:
#training on low resolution only G1
!python train.py --continue_train --label_nc 0 --no_instance --name nebbia_low --dataroot ./datasets/nebbia_low --resize_or_crop crop --fineSize 512 --batchSize 4

------------ Options -------------
batchSize: 4
beta1: 0.5
checkpoints_dir: ./checkpoints
continue_train: True
data_type: 32
dataroot: ./datasets/nebbia_low
debug: False
display_freq: 100
display_winsize: 512
feat_num: 3
fineSize: 512
fp16: False
gpu_ids: [0]
input_nc: 3
instance_feat: False
isTrain: True
label_feat: False
label_nc: 0
lambda_feat: 10.0
loadSize: 1024
load_features: False
load_pretrain: 
local_rank: 0
lr: 0.0002
max_dataset_size: inf
model: pix2pixHD
nThreads: 2
n_blocks_global: 9
n_blocks_local: 3
n_clusters: 10
n_downsample_E: 4
n_downsample_global: 4
n_layers_D: 3
n_local_enhancers: 1
name: nebbia_low
ndf: 64
nef: 16
netG: global
ngf: 64
niter: 100
niter_decay: 100
niter_fix_global: 0
no_flip: False
no_ganFeat_loss: False
no_html: False
no_instance: True
no_lsgan: False
no_vgg_loss: False
norm: instance
num_D: 2
output_nc: 3
phase: train
pool_size: 0
print_freq: 100
resize_or_crop: crop
save_epoch_freq: 10
save_latest_freq: 1000
serial_batches: False
tf_log: False
us

In [ ]:
#training on low resolution only G1
!python train.py --continue_train --label_nc 0 --no_instance --name nebbia_medium --dataroot ./datasets/nebbia_medium --resize_or_crop crop --fineSize 512 --batchSize 4

------------ Options -------------
batchSize: 4
beta1: 0.5
checkpoints_dir: ./checkpoints
continue_train: True
data_type: 32
dataroot: ./datasets/nebbia_medium
debug: False
display_freq: 100
display_winsize: 512
feat_num: 3
fineSize: 512
fp16: False
gpu_ids: [0]
input_nc: 3
instance_feat: False
isTrain: True
label_feat: False
label_nc: 0
lambda_feat: 10.0
loadSize: 1024
load_features: False
load_pretrain: 
local_rank: 0
lr: 0.0002
max_dataset_size: inf
model: pix2pixHD
nThreads: 2
n_blocks_global: 9
n_blocks_local: 3
n_clusters: 10
n_downsample_E: 4
n_downsample_global: 4
n_layers_D: 3
n_local_enhancers: 1
name: nebbia_medium
ndf: 64
nef: 16
netG: global
ngf: 64
niter: 100
niter_decay: 100
niter_fix_global: 0
no_flip: False
no_ganFeat_loss: False
no_html: False
no_instance: True
no_lsgan: False
no_vgg_loss: False
norm: instance
num_D: 2
output_nc: 3
phase: train
pool_size: 0
print_freq: 100
resize_or_crop: crop
save_epoch_freq: 10
save_latest_freq: 1000
serial_batches: False
tf_log: Fa

KeyboardInterrupt: ignored

In [ ]:
#training on low resolution only G1
!python train.py --continue_train --label_nc 0 --no_instance --name nebbia_high --dataroot ./datasets/nebbia_high --resize_or_crop crop --fineSize 512 --batchSize 4

------------ Options -------------
batchSize: 4
beta1: 0.5
checkpoints_dir: ./checkpoints
continue_train: True
data_type: 32
dataroot: ./datasets/nebbia_high
debug: False
display_freq: 100
display_winsize: 512
feat_num: 3
fineSize: 512
fp16: False
gpu_ids: [0]
input_nc: 3
instance_feat: False
isTrain: True
label_feat: False
label_nc: 0
lambda_feat: 10.0
loadSize: 1024
load_features: False
load_pretrain: 
local_rank: 0
lr: 0.0002
max_dataset_size: inf
model: pix2pixHD
nThreads: 2
n_blocks_global: 9
n_blocks_local: 3
n_clusters: 10
n_downsample_E: 4
n_downsample_global: 4
n_layers_D: 3
n_local_enhancers: 1
name: nebbia_high
ndf: 64
nef: 16
netG: global
ngf: 64
niter: 100
niter_decay: 100
niter_fix_global: 0
no_flip: False
no_ganFeat_loss: False
no_html: False
no_instance: True
no_lsgan: False
no_vgg_loss: False
norm: instance
num_D: 2
output_nc: 3
phase: train
pool_size: 0
print_freq: 100
resize_or_crop: crop
save_epoch_freq: 10
save_latest_freq: 1000
serial_batches: False
tf_log: False


KeyboardInterrupt: ignored

# Pix2Pix test


## General model epoch 50


In [26]:
#se non è stato eseguito il download
if os.getcwd() != "/content/drive/MyDrive/Haze-Fog-suppression/pix2pixHD":
  os.chdir("pix2pixHD")

!python test.py --dataroot ./datasets/indoor --name nebbia --resize_or_crop none --label_nc 0 --no_instance --which_epoch 50 --results_dir ./datasets/indoor/synth
!python test.py --dataroot ./datasets/outdoor --name nebbia --resize_or_crop none --label_nc 0 --no_instance --which_epoch 50 --results_dir ./datasets/outdoor/synth

------------ Options -------------
aspect_ratio: 1.0
batchSize: 1
checkpoints_dir: ./checkpoints
cluster_path: features_clustered_010.npy
data_type: 32
dataroot: ./datasets/indoor
display_winsize: 512
engine: None
export_onnx: None
feat_num: 3
fineSize: 512
fp16: False
gpu_ids: [0]
how_many: 50
input_nc: 3
instance_feat: False
isTrain: False
label_feat: False
label_nc: 0
loadSize: 1024
load_features: False
local_rank: 0
max_dataset_size: inf
model: pix2pixHD
nThreads: 2
n_blocks_global: 9
n_blocks_local: 3
n_clusters: 10
n_downsample_E: 4
n_downsample_global: 4
n_local_enhancers: 1
name: nebbia
nef: 16
netG: global
ngf: 64
niter_fix_global: 0
no_flip: False
no_instance: True
norm: instance
ntest: inf
onnx: None
output_nc: 3
phase: test
resize_or_crop: none
results_dir: ./datasets/indoor/synth
serial_batches: False
tf_log: False
use_dropout: False
use_encoded_image: False
verbose: False
which_epoch: 50
-------------- End ----------------
CustomDatasetDataLoader
dataset [AlignedDataset] 

In [30]:
# Definisci i percorsi delle cartelle contenenti le immagini
indoor_synth_folder = "/content/drive/MyDrive/Haze-Fog-suppression/pix2pixHD/datasets/indoor/synth/nebbia/test_50/images"  # Cartella con le immagini generate
indoor_original_folder = "/content/drive/MyDrive/Haze-Fog-suppression/pix2pixHD/datasets/indoor/test_A"        # Cartella con le immagini originali
outdoor_synth_folder = "/content/drive/MyDrive/Haze-Fog-suppression/pix2pixHD/datasets/outdoor/synth/nebbia/test_50/images"  # Cartella con le immagini generate
outdoor_original_folder = "/content/drive/MyDrive/Haze-Fog-suppression/pix2pixHD/datasets/outdoor/test_A"       # Cartella con le immagini originali
result_folder = "/content/drive/MyDrive/Haze-Fog-suppression/pix2pixHD/results"

test_indoor_outdoor(indoor_synth_folder, indoor_original_folder, outdoor_synth_folder, outdoor_original_folder,os.path.join(result_folder,"50"))

AttributeError: ignored

## General Model full train epoch 110

In [ ]:
!python test.py --dataroot ./datasets/outdoor --name nebbia --resize_or_crop none --label_nc 0 --no_instance --results_dir ./datasets/outdoor/synth

In [ ]:
# Definisci i percorsi delle cartelle contenenti le immagini
indoor_synth_folder = "/content/drive/MyDrive/Haze-Fog-suppression/pix2pixHD/datasets/indoor/synth/nebbia/test_latest/images"  # Cartella con le immagini generate
outdoor_synth_folder = "/content/drive/MyDrive/Haze-Fog-suppression/pix2pixHD/datasets/outdoor/synth/nebbia/test_latest/images"  # Cartella con le immagini generate

test_indoor_outdoor(indoor_synth_folder, indoor_original_folder, outdoor_synth_folder, outdoor_original_folder,os.path.join(result_folder,"latest"))

## Classifier + 3GAN epoch 50

### Classifier

In [15]:
from keras.models import load_model
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import cv2
from google.colab.patches import cv2_imshow

# Carica il tuo modello di classificazione
classification_model = load_model("./classifier/dataset/models/InceptionV3/InceptionV3_128.h5")
#classification_model = load_model('./classifier/dataset/models/VGG16/VGG16_model.h5')

# Definisci le etichette delle classi
class_labels = ["no_nebbia", "nebbia_low", "nebbia_medium", "nebbia_high"]

# Percorso delle immagini di input
base_path = "./pix2pixHD/datasets/"
context = ["indoor","outdoor"]

# Percorso della cartella dei risultati
output_folder = "./pix2pixHD/results/"
for scene in context:
  input_folder = os.path.join(base_path, os.path.join(scene, "test_A"))
  # Ciclo attraverso tutte le immagini nella cartella di input
  for filename in os.listdir(input_folder):
    if filename.endswith(".png") or filename.endswith(".jpg"):
      # Carica l'immagine di input
      input_image = cv2.imread(os.path.join(input_folder, filename))

      # Ridimensiona l'immagine
      target_width = 512
      target_height = 512
      resized_image = Image.fromarray(input_image).resize((target_width, target_height))
      resized_image_array = np.array(resized_image)
      resized_image_array = np.expand_dims(resized_image_array, axis=0)

      # Esegui la classificazione dell'immagine
      predicted_probabilities = classification_model.predict(resized_image_array)
      predicted_class = np.argmax(predicted_probabilities)
      image_class_label = class_labels[predicted_class]

      # Crea la cartella di output se non esiste
      output_class_folder = os.path.join(output_folder, os.path.join("nebbia3GAN",os.path.join(scene,os.path.join(image_class_label,"test_A"))))
      os.makedirs(output_class_folder, exist_ok=True)

      # Salva l'immagine classificata nella cartella di output
      output_path = os.path.join(output_class_folder, filename)
      cv2.imwrite(output_path, input_image)



1/1 [==============================] - 0s 40ms/step


### Generate Images

In [20]:
import subprocess

class_labels = ["nebbia_low", "nebbia_medium", "nebbia_high"]
context = ["indoor","outdoor"]
base_path = "./pix2pixHD/results/nebbia3GAN"
checkpoints_dir = "./pix2pixHD/checkpoints"

for scene in context:
  for label in class_labels:
    dataset_path = os.path.join(base_path, os.path.join(scene, label))
    result_path = os.path.join(base_path, os.path.join(scene, os.path.join(label, "synth")))
    # Definisci il comando da eseguire
    command = [
        'python', 'pix2pixHD/test.py',
        '--dataroot', dataset_path,
        '--checkpoints_dir', checkpoints_dir,
        '--name', label,
        '--resize_or_crop', 'none',
        '--label_nc', '0',
        '--no_instance',
        '--which_epoch', '50',
        '--results_dir', result_path
    ]

    # Esegui il comando utilizzando subprocess
    completed_process = subprocess.run(command, stdout=subprocess.PIPE, text=True)

    # Ottieni l'output dall'oggetto CompletedProcess
    output = completed_process.stdout

    # Stampa l'output
    print(label)
    print(output)

nebbia_low
------------ Options -------------
aspect_ratio: 1.0
batchSize: 1
checkpoints_dir: ./pix2pixHD/checkpoints
cluster_path: features_clustered_010.npy
data_type: 32
dataroot: ./pix2pixHD/results/nebbia3GAN/indoor/nebbia_low
display_winsize: 512
engine: None
export_onnx: None
feat_num: 3
fineSize: 512
fp16: False
gpu_ids: [0]
how_many: 50
input_nc: 3
instance_feat: False
isTrain: False
label_feat: False
label_nc: 0
loadSize: 1024
load_features: False
local_rank: 0
max_dataset_size: inf
model: pix2pixHD
nThreads: 2
n_blocks_global: 9
n_blocks_local: 3
n_clusters: 10
n_downsample_E: 4
n_downsample_global: 4
n_local_enhancers: 1
name: nebbia_low
nef: 16
netG: global
ngf: 64
niter_fix_global: 0
no_flip: False
no_instance: True
norm: instance
ntest: inf
onnx: None
output_nc: 3
phase: test
resize_or_crop: none
results_dir: ./pix2pixHD/results/nebbia3GAN/indoor/nebbia_low/synth
serial_batches: False
tf_log: False
use_dropout: False
use_encoded_image: False
verbose: False
which_epoch: 5

### Evaluate images

In [24]:
import shutil

class_labels = ["nebbia_low", "nebbia_medium", "nebbia_high"]
context = ["indoor","outdoor"]
base_path = "/content/drive/MyDrive/Haze-Fog-suppression/pix2pixHD/results/nebbia3GAN/"
for scene in context:
  for label in class_labels:
    source = os.path.join(base_path, os.path.join(scene, os.path.join(label, os.path.join("synth", os.path.join(label, os.path.join("test_50", "images"))))))
    dest = os.path.join(base_path, os.path.join(scene, "synth"))

    # Ottieni la lista dei file nella cartella di origine
    elenco_file = os.listdir(source)

    if not os.path.exists(dest):
      os.makedirs(dest)

    # Loop attraverso tutti i file nella cartella di origine e spostali nella cartella di destinazione
    for file in elenco_file:
        percorso_file_origine = os.path.join(source, file)
        percorso_file_destinazione = os.path.join(dest, file)

        # Sposta il file
        shutil.move(percorso_file_origine, percorso_file_destinazione)

print("Spostamento completato.")

Spostamento completato.


In [ ]:
# Definisci i percorsi delle cartelle contenenti le immagini
indoor_synth_folder = "/content/drive/MyDrive/Haze-Fog-suppression/pix2pixHD/results/nebbia3GAN/indoor/synth"  # Cartella con le immagini generate
outdoor_synth_folder = "/content/drive/MyDrive/Haze-Fog-suppression/pix2pixHD/results/nebbia3GAN/outdoor/synth"  # Cartella con le immagini generate

test_indoor_outdoor(indoor_synth_folder, indoor_original_folder, outdoor_synth_folder, outdoor_original_folder,os.path.join(result_folder,"3GAN"))